In [92]:
import pandas as pd
import numpy as np

In [93]:
# load the materialized benchmark config
stats_file = "../results/combined.stats.csv"
sim_df = pd.read_csv(stats_file, header=0)
assert((sim_df["input_mode"] == "serial").sum() > 0)
sim_df

,cycles,instructions,num_blocks,warp_inst,exec_time_sec,is_release_build,dram_reads,dram_writes,dram_accesses,l2_reads,...,input_mode,input_threads,input_run_ahead,target,benchmark,input_id,l2_read_hit_rate,l2_write_hit_rate,l2_read_miss_rate,l2_write_miss_rate
0,1117.0,10516.0,1.0,0.00000,0.044,True,56,13,69.0,10,...,serial,4.0,5.0,Simulate,vectorAdd,0,NaN,NaN,NaN,NaN
1,1117.0,10516.0,1.0,0.00000,0.054,True,56,13,69.0,10,...,serial,4.0,5.0,Simulate,vectorAdd,0,NaN,NaN,NaN,NaN
2,1117.0,10516.0,1.0,0.00000,0.070,True,56,13,69.0,10,...,serial,4.0,5.0,Simulate,vectorAdd,0,NaN,NaN,NaN,NaN
3,1117.0,10516.0,1.0,0.00000,0.051,True,56,13,69.0,10,...,serial,4.0,5.0,Simulate,vectorAdd,0,NaN,NaN,NaN,NaN
4,1117.0,10516.0,1.0,0.00000,0.150,True,56,13,69.0,10,...,deterministic,4.0,5.0,Simulate,vectorAdd,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,1648.0,22216.0,1.0,23.00000,0.000,True,392,125,517.0,66,...,NaN,NaN,NaN,PlaygroundSimulate,vectorAdd,1,NaN,NaN,NaN,NaN
236,2139.0,222160.0,10.0,22.69375,0.000,True,3764,1250,5014.0,628,...,NaN,NaN,NaN,PlaygroundSimulate,vectorAdd,2,NaN,NaN,NaN,NaN
237,1331.0,10516.0,1.0,10.75000,0.000,True,92,25,117.0,16,...,NaN,NaN,NaN,PlaygroundSimulate,vectorAdd,3,NaN,NaN,NaN,NaN
238,2037.0,22216.0,1.0,23.00000,0.000,True,764,250,1014.0,128,...,NaN,NaN,NaN,PlaygroundSimulate,vectorAdd,4,NaN,NaN,NaN,NaN


In [94]:
# ideas for stats

# overall correlation per target a) with native b) with accelsim
# all targets: bar plots with the per benchmark and target
# table: error when using parallel execution, and speedup

In [95]:
stat_cols = [
    "exec_time_sec",
    "cycles",
    "num_blocks",
    "instructions",
    "warp_inst",
    "dram_reads",
    "dram_writes",
    "l2_accesses",
    "l2_read_hit_rate",
    "l2_write_hit_rate",
    "l2_read_miss_rate",
    "l2_write_miss_rate",
    "l2_read_hits",
    "l2_write_hits",
    "l2_read_misses",
    "l2_write_misses",
    "l2_hits",
    "l2_misses",
]

index_cols = ["target", "benchmark", "input_id"]

In [96]:
# mean stddev, rel err, mae, rmse for native profiling per metric
benchmarks = ["vectorAdd"]

selected_df = sim_df.copy()
selected_df = selected_df[selected_df["target"] == "Profile"]
if len(benchmarks) > 0:
    selected_df = selected_df[selected_df["benchmark"].isin(benchmarks)]


# assert((selected_df["is_release_build"] == True).all())
# print(selected_df[stat_cols])

# target	benchmark
# [["benchmark", "target", "cycles", "kernel_name"]]
grouped = selected_df.groupby(index_cols)
# grouped[["cycles", "instructions"]].std()
grouped[stat_cols].std()
# selected_df

exec_time_sec       cycles  num_blocks  \
target  benchmark input_id                                           
Profile vectorAdd 0              0.139839   698.205062         0.0   
                  1              0.179013  2841.638484         0.0   
                  2              0.083948  1020.049725         0.0   
                  3              0.062288   686.155253         0.0   
                  4              0.125004   721.782593         0.0   
                  5              0.859590  1478.956008         0.0   

                            instructions  warp_inst  dram_reads  dram_writes  \
target  benchmark input_id                                                     
Profile vectorAdd 0                  0.0        0.0  181.876759     7.306010   
                  1                  0.0        0.0  312.638165    18.590619   
                  2                  0.0        0.0  487.289658   181.137425   
                  3                  0.0        0.0   92.034052    15.317383   
                  4                  0.0        0.0  858.194358    92.222195   
                  5                  0.0        0.0  147.468867   349.254142   

                            l2_accesses  l2_read_hit_rate  l2_write_hit_rate  \
target  benchmark input_id                                                     
Profile vectorAdd 0          235.945286          0.000000                0.0   
                  1          242.693588          0.316228                0.0   
                  2          276.221248          0.000000                0.0   
                  3          127.037002          0.000000                0.0   
                  4          198.508914          0.000000                0.0   
                  5          208.932631          0.000000                0.0   

                            l2_read_miss_rate  l2_write_miss_rate  \
target  benchmark input_id                                          
Profile vectorAdd 0                  0.000000                 0.0   
                  1                  0.316228                 0.0   
                  2                  0.000000                 0.0   
                  3                  0.000000                 0.0   
                  4                  0.000000                 0.0   
                  5                  0.000000                 0.0   

                            l2_read_hits  l2_write_hits  l2_read_misses  \
target  benchmark input_id                                                
Profile vectorAdd 0             0.000000            0.0      236.524746   
                  1           265.315096            0.0      253.311161   
                  2             0.000000            0.0      273.948981   
                  3             0.000000            0.0      118.810774   
                  4             0.000000            0.0      190.945047   
                  5             0.000000            0.0      203.085204   

                            l2_write_misses  l2_hits      l2_misses  
target  benchmark input_id                                           
Profile vectorAdd 0               13.063945      0.0   10375.039268  
                  1               13.492385      0.0   43843.259646  
                  2               14.009521      0.0  357727.093067  
                  3               10.119289      0.0    9387.528237  
                  4               17.281975      0.0   61996.862602  
                  5               16.949598      0.0  552722.893905

In [97]:
# error table for metrics
benchmarks = ["vectorAdd"]
# targets = ["Simulate"]
selected_df = sim_df.copy()
if len(benchmarks) > 0:
    selected_df = selected_df[selected_df["benchmark"].isin(benchmarks)]
# only compare serial gpucachesim
# selected_df = selected_df[selected_df["input_mode"] != "nondeterministic"]
# selected_df = selected_df[selected_df["target"].isin(targets)]]
# print(selected_df[selected_df["target"] == "Simulate"])
# print(selected_df["target"].unique().tolist())

# assert((selected_df["is_release_build"] == True).all())

# target	benchmark
# [["benchmark", "target", "cycles", "kernel_name"]]
test = selected_df.groupby(index_cols)[["cycles", "instructions"]].mean().reset_index()
# print(test)

test.pivot(index=["benchmark", "input_id"], columns='target', values=['cycles', "instructions"])

cycles                                        \
target             AccelsimSimulate PlaygroundSimulate   Profile Simulate   
benchmark input_id                                                          
vectorAdd 0                  1285.0             1285.0  6450.150   1117.0   
          1                  1648.0             1648.0  8262.700   1117.0   
          2                  2139.0             2139.0  7161.700   1117.0   
          3                  1331.0             1331.0  6567.650   1125.0   
          4                  2037.0             2037.0  7867.925   1130.0   
          5                  3397.0             3397.0  9469.550   1125.0   
          6                     NaN                NaN       NaN   1130.0   
          7                     NaN                NaN       NaN   1228.0   
          8                     NaN                NaN       NaN   1228.0   
          9                     NaN                NaN       NaN   1228.0   
          10                    NaN                NaN       NaN   1235.0   
          11                    NaN                NaN       NaN   1240.0   
          12                    NaN                NaN       NaN   1235.0   
          13                    NaN                NaN       NaN   1240.0   
          14                    NaN                NaN       NaN   1311.0   
          15                    NaN                NaN       NaN   1311.0   
          16                    NaN                NaN       NaN   1311.0   
          17                    NaN                NaN       NaN   1315.0   
          18                    NaN                NaN       NaN   1320.0   
          19                    NaN                NaN       NaN   1315.0   
          20                    NaN                NaN       NaN   1320.0   
          21                    NaN                NaN       NaN   1119.0   
          22                    NaN                NaN       NaN   1119.0   
          23                    NaN                NaN       NaN   1119.0   
          24                    NaN                NaN       NaN   1125.0   
          25                    NaN                NaN       NaN   1130.0   
          26                    NaN                NaN       NaN   1125.0   
          27                    NaN                NaN       NaN   1130.0   
          28                    NaN                NaN       NaN   1267.0   
          29                    NaN                NaN       NaN   1267.0   
          30                    NaN                NaN       NaN   1267.0   
          31                    NaN                NaN       NaN   1275.0   
          32                    NaN                NaN       NaN   1280.0   
          33                    NaN                NaN       NaN   1275.0   
          34                    NaN                NaN       NaN   1280.0   
          35                    NaN                NaN       NaN   1588.0   
          36                    NaN                NaN       NaN   1588.0   
          37                    NaN                NaN       NaN   1588.0   
          38                    NaN                NaN       NaN   1595.0   
          39                    NaN                NaN       NaN   1600.0   
          40                    NaN                NaN       NaN   1595.0   
          41                    NaN                NaN       NaN   1600.0   

                       instructions                                         
target             AccelsimSimulate PlaygroundSimulate   Profile  Simulate  
benchmark input_id                                                          
vectorAdd 0                 10516.0            10516.0   10516.0   10516.0  
          1                 22216.0            22216.0   22216.0   10516.0  
          2                222160.0           222160.0  222160.0   10516.0  
          3                 10516.0            10516.0   10516.0   10516.0  
          4             

In [98]:
# bar plots for cycles

benchmarks = ["vectorAdd"]
targets = ["Simulate"]
selected_df = sim_df.copy()
selected_df = selected_df[selected_df["benchmark"].isin(benchmarks)]
selected_df = selected_df[selected_df["target"].isin(targets)]

assert((selected_df["is_release_build"] == True).all())

In [129]:
# relative error for nondeterministic gpucachesim
benchmarks = ["vectorAdd"]
targets = ["Simulate"]
selected_df = sim_df.copy()
selected_df = selected_df[selected_df["benchmark"].isin(benchmarks)]
selected_df = selected_df[selected_df["target"].isin(targets)]

assert((selected_df["is_release_build"] == True).all())

serial = selected_df[selected_df["input_mode"] == "serial"]
nondet = selected_df[selected_df["input_mode"] == "nondeterministic"]
print((nondet["input_id"] == 3).sum())
print(nondet[nondet["input_id"] == 41][["cycles", "l2_reads", "dram_accesses"]])
print(len(serial))
print(len(nondet))

# those are fully distinct
print(serial["input_id"].unique())
print(nondet["input_id"].unique())

# print(serial[[c for c in serial.columns if c.startswith('input_')]])
# print(nondet[[c for c in nondet.columns if c.startswith('input_')]])
bench_cols = ["benchmark"]
input_cols = ["input_dtype", "input_length"]
# selected_df[["benchmark", "target", "cycles"]]

# join based on input_cols
joined = nondet.merge(serial, on=bench_cols+ input_cols, how="left", suffixes=('_nondet', '_serial'))
# selected_df
# nondet
joined = joined[joined["input_id_nondet"] == 41]
joined[[
    "benchmark", "input_dtype", "input_length", "input_id_nondet",  "input_id_serial",
    "input_threads_nondet", "input_run_ahead_nondet",
    "exec_time_sec_nondet", "exec_time_sec_serial",
    "cycles_nondet", "cycles_serial"
]]

4
     cycles  l2_reads  dram_accesses
164  1600.0      1252        14440.0
165  1600.0      1252        14440.0
166  1600.0      1252        14440.0
167  1600.0      1252        14440.0
24
96
[ 0  7 14 21 28 35]
[ 3  4  5  6 10 11 12 13 17 18 19 20 24 25 26 27 31 32 33 34 38 39 40 41]


,benchmark,input_dtype,input_length,input_id_nondet,input_id_serial,input_threads_nondet,input_run_ahead_nondet,exec_time_sec_nondet,exec_time_sec_serial,cycles_nondet,cycles_serial
368,vectorAdd,64,10000,41,35,8.0,10.0,0.814,0.871,1600.0,1588.0
369,vectorAdd,64,10000,41,35,8.0,10.0,0.814,0.706,1600.0,1588.0
370,vectorAdd,64,10000,41,35,8.0,10.0,0.814,0.691,1600.0,1588.0
371,vectorAdd,64,10000,41,35,8.0,10.0,0.814,0.757,1600.0,1588.0
372,vectorAdd,64,10000,41,35,8.0,10.0,0.868,0.871,1600.0,1588.0
373,vectorAdd,64,10000,41,35,8.0,10.0,0.868,0.706,1600.0,1588.0
374,vectorAdd,64,10000,41,35,8.0,10.0,0.868,0.691,1600.0,1588.0
375,vectorAdd,64,10000,41,35,8.0,10.0,0.868,0.757,1600.0,1588.0
376,vectorAdd,64,10000,41,35,8.0,10.0,0.884,0.871,1600.0,1588.0
377,vectorAdd,64,10000,41,35,8.0,10.0,0.884,0.706,1600.0,1588.0


In [100]:
# gpusims.plot.metrics.Cycles: plot_scatter,
fig = metrics_plot_func(
    data=per_config_metric_df,
    config=config,
    metric_cls=metric_cls,
    title=f"{metric_cls.name} Correlation ({config.name})",
)
# filename = ["scatter", metric_cls.name, config.key]
# filename = Path("./figs") / gpusims.utils.slugify("_".join(filename))
# filename = filename.with_suffix(".pdf")
# fig.write_image(filename, **PDF_OPTS)
fig.show()

NameError: name 'metrics_plot_func' is not defined

In [101]:
for metric_cls, metrics_plot_func in metrics.items():
    print(metric_cls.name)
    all_metric_df = []
    # for (config_name, config) in selected_configs_copy.items():
    for config_name in selected_configs_copy:
        config = configs[config_name]
        per_config_metric_df = []
        
        # for (bench_name, bench) in selected_benchmarks_copy.items():
        for bench_name, selected_bench_inputs in selected_benchmarks_copy:
            bench = benchmarks[bench_name]
            supported_simulators = [
                sim_name for sim_name in selected_simulators
                if bench.enabled(sim_name) # and inp.enabled(sim_name)
            ]
            # for inp in bench.inputs:
            for inp_args, inp_abbr in selected_bench_inputs:
                # print(config_name, bench_name, inp)
                
                inp = next(i for i in bench.inputs if i.args.strip() == inp_args.strip())
                assert inp is not None, f"input {inp_args} does not exist"
                
                plot_data = PlotData(benchmark=bench, config=config, inp=inp)
                # for (sim_name, sim) in selected_simulators_copy.items():
                for sim_name in supported_simulators:
                    sim = simulators[sim_name]
                    
                    if not bench.enabled(sim_name):
                        continue
                    if not inp.enabled(sim_name):
                        continue
                    # print(sim_name, config_name, bench_name)
                    bench_config = sim(
                        run_dir=run_dir / sim_name.lower(),
                        benchmark=bench,
                        config=config,
                    )
                    if not bench_config.input_path(inp).is_dir():
                        print(f"WARN: {bench_config.input_path(inp)} does not exist")
                        continue

                    plot_data[sim_name] = bench_config.load_dataframe(inp)

                metric = metric_cls(plot_data)
                metric_df = metric.compute()
                metric_df["Benchmark"] = f"{bench.name}<br>{inp.args}"
                metric_df["Config"] = config_name

                per_config_metric_df.append(metric_df)
                all_metric_df.append(metric_df)

        per_config_metric_df = pd.concat(per_config_metric_df)
        # break
        # continue
        
        fig = metrics_plot_func(
            data=per_config_metric_df,
            config=config,
            metric_cls=metric_cls,
            title=f"{metric_cls.name} Correlation ({config.name})",
        )
        filename = ["scatter", metric_cls.name, config.key]
        filename = Path("./figs") / gpusims.utils.slugify("_".join(filename))
        filename = filename.with_suffix(".pdf")
        fig.write_image(filename, **PDF_OPTS)
        print("wrote", filename)
    
    all_metric_df = pd.concat(all_metric_df)
    if True:
        fig = plot_scatter_subplots(
            in_data=all_metric_df,
            metric_cls=metric_cls,
            title=f"{metric_cls.name} Correlation",
        )
        filename = ["per-configs-scatter", metric_cls.name]
        filename = Path("./figs") / gpusims.utils.slugify("_".join(filename))
        filename = filename.with_suffix(".pdf")
        fig.write_image(filename, **PDF_OPTS)
        print("wrote", filename)
        
    fig = metrics_plot_func(
        data=all_metric_df,
        config=config,
        metric_cls=metric_cls,
        title=f"{metric_cls.name} Correlation",
    )
    filename = ["all-configs-scatter", metric_cls.name]
    filename = Path("./figs") / gpusims.utils.slugify("_".join(filename))
    filename = filename.with_suffix(".pdf")
    fig.write_image(filename, **PDF_OPTS)
    print("wrote", filename)

# all_metric_df

NameError: name 'metrics' is not defined